In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import matplotlib.pylab as pylab
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_iris
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2


pylab.rcParams['figure.figsize'] = 5, 5
plt.rc("font", size=15)


plt.rc('xtick', color='k', labelsize='medium', direction='in')
plt.rc('xtick.major', size=8, pad=12)
plt.rc('xtick.minor', size=8, pad=12)

plt.rc('ytick', color='k', labelsize='medium', direction='in')
plt.rc('ytick.major', size=8, pad=12)
plt.rc('ytick.minor', size=8, pad=12)

In this notebook we are going to learn what a **decision tree** is, how it does classification and what the **hyperparameters** used to define it actually do and stand for.

Then we are going to superficially describe how decision trees are basic bricks of more powerful algorithms called **ensemble algorithms**. You will see that these algorithm can be quite diverse. Since they are quite useful and commonly used I will spend some time showing how different they actually are from one another and what that means. 

# Decision tree based classification

## Table Of Content:<a id='toc'></a>

* [Decision Tree](#Decision-Tree)
    * [Toy data : exploring hyperparameters](#DT1)
    * [Decision tree on the Cancer data set](#DTcancer)
    * [Exercise : Decision tree](#DTexo)
* [Random Forest](#Random-Forest)
    * [Random forest on the Cancer data set](#RFcancer)
* [Exercise](#exo)


* [Appendices](#APPENDIX)
    * [Ada Boost](#Ada-Boost)
    * [Boosted Gradient](#Boosted-Gradient)




In [ ]:
# the breast cancer dataset is integrated in the sklearn library
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

## we reduce the features because otherwise this problem is a bit too easy ;-)
m = list( map( lambda x : x.startswith("mean ") , data["feature_names"] ) )


X_cancer=data['data'][:,m]

# for some reason this dataset has encoded 0 for malignant,
# which is extremely counter intuitive
# So I choose to switch it to the more intuitive order now
y_cancer= 1-data['target']

#making it into a dataframe
df_cancer=pd.DataFrame(X_cancer,
    columns=data["feature_names"][m])

df_cancer["malignant"]=y_cancer

# 0 : benign
# 1 : malignant


df_cancer.head()

In [ ]:

X_cancer_train, X_cancer_test, y_cancer_train, y_cancer_test = train_test_split(X_cancer, y_cancer,
                                                   random_state=7, stratify=y_cancer)



[Backto the ToC](#toc)

# Decision tree <a class="anchor" id="Decision-Tree"></a>

A simple **decision tree** reduces your problem into a **hierarchichal sequence of questions** on your features that can be answered by yes or no and which subdivides the data into 2 subgroups on which a new question is asked, and so on and so on.

![tree_ex](../images/tree_ex.png)

Ok, but a huge number of trees can actually be built just by considering the different orders of questions asked. How does the algorithm deals with this?

Quite simply actually: it **tests all the features and chooses the most discriminative** (with respect to your target variable) : the feature where a yes or no question divides the data into 2 subsets which minimizes an **impurity measure**.

Imagine you have a dataset with feature color (red or blue) and feature shape (square or circle), and 2 target classes : 1 and 2.

![tree](../images/Tree.png)

Asking `"feature color is red"` gives you the following subgroups:
 * 10 class 1, and 1 class 2 (`"feature color is red" == True`)
 * 2 class 1, and 11 class 2 (`"feature color is red" == False`)

Asking `"feature shape is square"` gives you:
 * 5 class 1, and 7 class 2 (`True`) 
 * 7 class 1 and 5 class 2 (`False`)
 
 So, you will prefer asking `"feature color is red?"` over `"feature shape is square?"`: `"feature color is red?"` is more discriminative.

For categorical variables, the questions test for a specific category.
For **numerical variables, the questions use a threshold** to as a yes/no question.  

The **threshold is, again, chosen to minimize impurity**. And in turn the best threshold for a variable is used to estimate the discriminativeness of that variable.

Of course, you will have to compute this threshold at each step of your tree since at each step you are considering different subdatasets.

---

The **impurity is related to how much your feature splitting is still having mixed classes**. So the impurity ends up giving a score: either it is a simple [Shannon entropy](https://en.wikipedia.org/wiki/Entropy_(information_theory)) or it is a [Gini coefficient](https://en.wikipedia.org/wiki/Gini_coefficient).

#### Shannon Entropy

$$Entropy = - \sum_{j} p_j log_2(p_j)$$

This measure is linked to information theory, where the information of an event occuring is the $log_2$ of this event's probability of occuring.
For purity, **0 is the best possible score, and 1 the worst**.

#### Gini coefficient

$$Gini = 1- \sum_{j} p_j^2$$

The idea is to measure the **probability that a dummy classifier mislabels your data**.
**0 is best, 1 is worst.**

---

Before going further, just a little bit of vocabulary: 
* **Trees** are made of **nodes** (where the question is asked and where the splitting occurs). 
* A **branch** is the outcome of a splitting. 
* A **leaf** is the last node on a branch (no more splitting).

[Backto the ToC](#toc)

## Toy data : exploring hyperparameters <a id="DT1"></a>


Let explore some hyperparameters of this method that, you will see in those examples, act like a regularization:
- **Max Tree depth**: the maximum number of consecutive questions to ask
- **Min Splitting of nodes**: minimum number of points to consider to make a new rule, outside of the leaves
- **Min Splitting of leaves**: minimum number of points to consider to make a new rule, at the leaves

In [ ]:
from sklearn.datasets import make_blobs
    
# 120 points, 3 blobs/clusters with some spread=3
blob_centers = np.array([[-7,2.5],[6,-10],[8,-3]])
blob_stds = [[1,3],[3,6],[3,6]]
X_3, y_3 = make_blobs(n_samples = 250, 
                      centers = blob_centers,
                      cluster_std = blob_stds, random_state = 42)


plt.scatter(X_3[:,0],X_3[:,1],c=y_3,cmap=plt.cm.coolwarm,edgecolors='k')

In [ ]:
from sklearn.tree import DecisionTreeClassifier
## creating a decision tree with 1 parameter changed (more on that later)
tree = DecisionTreeClassifier(max_depth=3)
tree.fit(X_3, y_3)

pd.crosstab( tree.predict( X_3 ) , y_3 , rownames=['truth'] , colnames=['prediction'] )

In [ ]:
from sklearn.tree import plot_tree
fig,ax = plt.subplots(figsize=(14,6))

_ = plot_tree( tree , feature_names=['x','y'] , 
               fontsize=14 , filled=True , impurity=False , precision=3, ax=ax)

In [ ]:
from utils import contour_tree

contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = None,
              min_s = 2,
              min_l = 1,
              max_f = None)
#You can see that there are 5 hyperparameters here. Let's see what they do and what they mean.
#I bet you can already guess it is going to be related to regularization....
# After X,y you have 
# * crit = 'entropy' which is one way to calculate impurity (you could also put gini here)
# * maxd : the max depth of your tree
# * min_s : the number of points that should be concerned by the making of a new rule (splitting of the nodes)
# * min_l : #of points that should be considered to make a final leaf classification
# * max_f : maximum number of features to consider for making a new rule...

Well that's a really complicated and zealous tree. Look at those boundaries. Clearly if we just let it go like that it is overfitting...

In [ ]:
#using another impurity measurement
contour_tree(X_3, y_3,
              crit = 'gini',
              maxd = None,
              min_s = 2,
              min_l = 1,
              max_f = None)

Still some overfitting but it is nice to see that the boundaries are different and that impurity calculations, even if very similar, are making a difference.

In [ ]:
#Imposing a limit for the depth of the tree : how many questions you ask (here set to 4)
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = 4,
              min_s = 2, min_l = 1, max_f = None)

Indeed only a maximum of 4 splitting events link the root to the leafs.

Here you stopped the tree at some points allowing some misslabelling instead of fitting perfectly the dataset.

In [ ]:
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = None,
              min_s = 2,
              min_l = 5,
              max_f = None)
# I don't really have a strong feeling about where I should stop the tree depth. 
# But I have an understanding of this other parameter (here set to 5) 
# called min_samples_leaf : 
#     it sets the minimal number of data points that the chain of rules should concern. 

# eg. Do you really wish to create a whole new set of rules to explain 
# only one particular data point? 

Convince yourself that now, rules are made for leafs of at least 4 points contrary to trees that we built before and for which you could have leaves containing only 2 points

The tree is simplified and the boundaries look more flexible.

In [ ]:
contour_tree(X_3, y_3,
              crit = 'entropy',
              maxd = None,
              min_s = 20,
              min_l = 2,
              max_f = None)
# Here it is the same as before but this time it applies to nodes instead of leaves
# This parameter is called min_samples_split and is set to 20 here.

Check that indeed there is no node splitting concerning less than 20 points

There are 3 main advantages to this kind of methods:
* it works with all types of feature
* you don't need to rescale
* it already includes non linear fitting

**Moreover it is 'easy' to interpret.**

But....(yes there is a but, there is no free lunch)

Even with all of those hyperparamaters **they are still not great on new data (inaccuracy...).** 

We will see that in the real data example below and we will see more powerful technics based on decision tree that are more costly but generalize better.

This hyperparameter set the number of randomly picked features to consider at each node splitting. By setting it up at `None` we consider all the features. But you could also fix that number as you wish or choose a known good scaling for your data which is either square root of the number of features or log2 of the number of features.

> N.B. this only affects node splitting : all the features are considered when the tree is built it is just that you randomly pick from the full set at each node splitting event.

---

We can also check the **effect of these parameters on the bias-variance tradeoff with a validation set**

In [ ]:
## and we create our own validation dataset
X_3_valid, y_3_valid = make_blobs(n_samples = 3000, 
                                  centers = blob_centers,
                                  cluster_std = blob_stds , random_state=654321)


In [ ]:
from sklearn.metrics import roc_auc_score

def test_decision_tree_param( X_train, y_train , X_valid , y_valid , param_name , param_range , ax):
    
    scores_valid = []
    scores_train = []

    for x in param_range:
        tree = DecisionTreeClassifier(**{param_name:x})
        tree.fit(X_3,y_3)

        y_train_prob = tree.predict_proba(X_train)
        y_valid_prob = tree.predict_proba(X_valid)

        scores_train.append(roc_auc_score(y_train , y_train_prob, multi_class='ovr'))
        scores_valid.append(roc_auc_score(y_valid , y_valid_prob, multi_class='ovr'))


    # plotting
    ax.plot(param_range, scores_train,'k-',linewidth=5,label='train')
    ax.plot(param_range, scores_valid,'r-',linewidth=5,label='valid')
    ax.set_xlabel(param_name)
    ax.set_ylabel('roc_auc')
    ax.set_title(param_name)
    ax.legend(loc='best')

In [ ]:
fig,ax = plt.subplots(1,3 , figsize = (15,5))

test_decision_tree_param( X_3, y_3 , X_3_valid , y_3_valid ,
                         param_name = "max_depth" , param_range=np.arange(1,20) , ax=ax[0])
test_decision_tree_param( X_3, y_3 , X_3_valid , y_3_valid ,
                         param_name = "min_samples_split" , param_range=np.arange(2,200) , ax=ax[1])
test_decision_tree_param( X_3, y_3 , X_3_valid , y_3_valid ,
                         param_name = "min_samples_leaf" , param_range=np.arange(1,200) , ax=ax[2])
fig.tight_layout()

<big>
    
 * **max_depth** : &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; &#8600; bias, &#8599; variance
 * **min_samples_split** : &#8599; bias, &#8600; variance
 * **min_samples_leaf** :&nbsp; &#8599; bias, &#8600; variance
 
</big>

[Back to ToC](#toc)

## Decision tree on the Cancer data set <a id='DTcancer'></a>

#### Let's train our model properly

In [ ]:
X_cancer_train.shape

In [ ]:
%%time
# define the hyperparameters you want to test
#with the range over which you want it to be tested.
grid_values = {'criterion': ['entropy','gini'],
               'max_depth':[2,6,10,14],
               'min_samples_split':np.arange(2,len(X_cancer_train)//4,5),
              'min_samples_leaf':np.arange(2,len(X_cancer_train)//4,5)}

grid_tree_roc_auc = GridSearchCV(DecisionTreeClassifier(class_weight='balanced'), 
                             param_grid = grid_values, 
                             cv=5,
                             scoring='roc_auc',n_jobs=-1)

grid_tree_roc_auc.fit(X_cancer_train, y_cancer_train)

print(f'Grid best score (max. {grid_tree_roc_auc.scoring}): {grid_tree_roc_auc.best_score_:.3f}')
print(f'Grid best parameter (max. {grid_tree_roc_auc.scoring}): ')
for k,v in grid_tree_roc_auc.best_params_.items():
      print('\t',k,'->',v)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix

y_test_score=grid_tree_roc_auc.score(X_cancer_test,y_cancer_test)

print('Grid best parameter (max. accuracy) model on test: ', y_test_score)

y_cancer_pred_test = grid_tree_roc_auc.predict(X_cancer_test)

confusion_m_cancer = confusion_matrix(y_cancer_test, y_cancer_pred_test)

plt.figure(figsize=(5.5,4))
sns.heatmap(confusion_m_cancer, annot=True)
plt.title('test {} : {:.3f}'.format( grid_tree_roc_auc.scoring , y_test_score ))
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
from sklearn.metrics import RocCurveDisplay

RocCurveDisplay.from_estimator(grid_tree_roc_auc,X_cancer_test, y_cancer_test)

That's a very complicated tree..... And not performing as well as you would expect from something so complicated....

#### feature importance

Feature importance in tree is calculated by checking the amount of impurity reduction link to a feature. We will use it but there are some problems in using that definition of feature importance (the main one being caridnality : if your features have a lot of possible value it becomes easier to find a good impurity splitting compared to a binary feature). A more robust way to do it would be to use permutation importance, which relies on looking at the effect of scrambling the values of a feature and see how much it impacts the model. Try it at some point, even on non tree algorithm [permutation_importance](https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html#sklearn.inspection.permutation_importance). 


In [ ]:
tree = grid_tree_roc_auc.best_estimator_

w=tree.feature_importances_#get the weights

featureW = pd.DataFrame( {'feature': df_cancer.columns[:-1],
                        'weight':w} )

# sort them by absolute value
featureWsorted = featureW.sort_values(by=['weight'] , 
                                      ascending=False , 
                                      key=lambda col : col.abs())

# get the non-null ones
print('Features sorted per importance:')
featureWsorted.loc[ featureWsorted["weight"] !=0 ]


In [ ]:
from sklearn.tree import plot_tree
fig,ax = plt.subplots(figsize=(25,10))
plot_tree( grid_tree_roc_auc.best_estimator_ , 
          feature_names=df_cancer.columns[:-1] , 
          ax=ax , fontsize=12 , filled=True , impurity=False , precision=3)
ax.set_title('best single decision tree')

[Back to the ToC](#toc)

##  Exercise : Decision tree <a id="DTexo"></a>

In [ ]:
## TBD ...

[Back to ToC](#toc)

# Random Forest <a class="anchor" id="Random-Forest"></a>

the Random Forest algorithm relies on two main concepts : 
1. **randomly producing/training many different trees**
2. **agglomerating the predictions** of all these trees (mainly averaging)


The randomness between trees concerns:
* **[bootstrapping](https://en.wikipedia.org/wiki/Bootstrapping_(statistics)) of the training dataset**
* using only a **random subset of features**


**Bootstrapping:** sampling methods in which you randomly draw a subsample from your data, *with replacement*. The created replicate is the same size as the original distribution.

I am sure you can see intuitively how that is going to help generalization of our model.

So now on top of all the parameters seen before to create each individual trees of the forest, you also have a parameter controlling the number of trees in your forest.


> In the following plots we will compare the results of a random forest with a single decision tree with the same hyperparameter values


![RF](../images/RF.png)

In [ ]:
from utils import contour_RF
contour_RF(X_3, y_3,
            n_tree = 10,
            crit = 'gini',
            maxd = 6,
            min_s = 10,
            min_l = 5,
            max_f = None)
#Same as for decision tree except that we have here one more hyperparameter, 
# here put to 100 and that represents the number of bootstraps 
# (number of trees trained and then participating to the vote)

See how in the case of random forest the iso-probability lines are less binary than in the case of the decision tree? 

In [ ]:
contour_RF(X_3, y_3,
            n_tree = 10, crit = 'gini',maxd = 6, min_s = 10, min_l = 5,
            max_f = 'sqrt')
# The difference here is that we set the max number of feature to split node to sqrt 
# (square root of the total number of features). 
#Before with None we were considering all the feature for node splitting.

In [ ]:
contour_RF(X_3, y_3,
            n_tree = 10, crit = 'gini',maxd = 2, min_s = 10, min_l = 5, max_f = 'sqrt')
# We change max depth from 6 to 2, 
# that would be equivalent to add some regularization.

We added a little bit of regularization in the single tree and so we can start to see less sharp borders. For the random forest it changes a bit the contours.

In [ ]:
contour_RF(X_3, y_3,
            n_tree = 1000, crit = 'gini',
            maxd = 3, 
            min_s = 10,
            min_l = 5,
            max_f = 'sqrt')
# with more trees we see more nuanced boundaries

[BAck to the ToC](#toc)

## Random forest on the Cancer data set <a id="RFcancer"></a>

In [ ]:
%%time
from sklearn.ensemble import RandomForestClassifier

grid_values = {'n_estimators' : [50,100,200], 
               'max_depth':np.arange(4,10), ## I reduce the search space in the interest of time too
               'min_samples_split':np.arange(2,12,2)}

grid_tree = GridSearchCV(RandomForestClassifier(class_weight='balanced'), 
                                param_grid = grid_values, 
                                scoring='roc_auc',
                                cv = 5,
                                n_jobs=-1)
grid_tree.fit(X_cancer_train, y_cancer_train)

print(f'Grid best score (roc_auc): {grid_tree.best_score_:.3f}')
print('Grid best parameter :')

for k,v in grid_tree.best_params_.items():
    print('{:>25}\t{}'.format(k,v))

In [ ]:
RF = grid_tree.best_estimator_

w=RF.feature_importances_#get the weights

featureW = pd.DataFrame( {'feature': df_cancer.columns[:-1] ,'weight':w} )

# sort them by absolute value
featureWsorted = featureW.sort_values(by=['weight'] , 
                                      ascending=False , 
                                      key=lambda col : col.abs())

# get the non-null ones
print('Features sorted per importance:')
featureWsorted.loc[ featureWsorted["weight"] !=0 ]

[Back to the ToC](#toc)

# Exercice : <a class="anchor" id="exo"></a>


[Acharjee et al.2016](https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-1043-4) propose several -omic dataset which they used to predict and gain knowledge on various phenotypic traits in potatos.

Here, we will concentrate on the their transcriptomics dataset and the phenotypic trait of the potato coloration

The data comes in an excel spreadsheet with multiple sheet.

Let's start with the metadata which contains our target variable :

In [ ]:
df = pd.read_excel('../data/12859_2016_1043_MOESM1_ESM.xlsx' )
df.index = df.Genotype
df.head()

## color is encoded between 0 and 50. 
# A threshold of 20 is good here to categorize between more white and more yellow potatoes
y = df["Flesh Colour"]>=20
y=y.replace({False:'White',True:'Yellow'})
y.value_counts()


Now let's read the sheet containing the transcriptomic data. Thankfully it has already been normalized so we only have minor adjustments to perform.

In [ ]:
xls = pd.ExcelFile('../data/12859_2016_1043_MOESM1_ESM.xlsx')
dfT = pd.read_excel(xls, 'Transcriptomics data ')

# transpose and keep only samples for which we have a target value
dfTT = dfT.T.loc[y.index , :]
dfTT.columns = dfT.loc[: , 'FeatureNum'].astype(str) 
dfTT.head()

Unfortunately, the gene column names are not very informative. According to the author, you can go to the supp. Mat. of another paper to get a (partial) annotation.

Your goal is to try to **predict potato color from the transcriptomic data** and, more importantly, to **retrieve which genes are the most important in this prediction**. Focus on the tree-based methods.

Here, the goal is not so much to get a classifier, but to use the process to gain knowledge on the molecular mechanisms which underly potato color. 

#### IMPORTANT CONSIDERATIONS

This dataset is, like many -omic dataset, fairly large (~15k genes have been quantified).
If you do not want to wait forever for your grid search to finish, we recommend you follow these suggestions :

 * start your pipeline with some feature selection, such as `SelectKBest` (in our test, 20 features was enough to get a good predictor)
 * max depth can be fixed at 2, min samples split at 2 and min samples leaf at 1
 * The total number of parameter combinations will control the time it takes:
     * 20 parameter combinations for random forest -> ~20s.
     * 25 parameter combinations for boosted gradient -> ~10s.
     * 25 parameter combinations for ADA boost -> ~20s.

For example, for a random forest:
```python
grid_values = {'classifier__criterion': ['entropy','gini'],
               'classifier__n_estimators':np.arange(1,1000,100), 
               'classifier__max_depth':[2,5],
               'classifier__min_samples_split':[2],
              'classifier__min_samples_leaf':[1]}
```
Represent : $2*10*2*1*1 = 40$ parameter combinations.

> with all these recommendations, the grid search computation on my personnal computer took about a minute



Correction : splitting the data

In [ ]:
# %load -r -13 solutions/solution_03_potato.py

Correction : pipeline and parameter space definition

In [ ]:
# %load -r 14-40 solutions/solution_03_potato.py

Correction : gridsearch on the pipeline

In [ ]:
# %load -r 41-51 solutions/solution_03_potato.py

Correction : reporting the best model fit

In [ ]:
# %load -r 52-77 solutions/solution_03_potato.py

Correction : getting the features with the most importance; ie. which genes are the best predictors for potato color ?

In [ ]:
# %load -r 78- solutions/solution_03_potato.py

[Back to ToC](#toc)

# Appendices <a class="anchor" id="APPENDIX"></a>

[Back to ToC](#toc)

# ADA Boost <a class="anchor" id="Ada-Boost"></a>

As in Random Forest, you have a forest of trees, or rather, of *stumps*. However:

* ADA boost considers **only the simplest type of trees : a single node with a yes or no question on one feature.** 
* Also, **in ADA boost stumps have different contributions in the final decision.** 
* Stumps are linked and there is an **order and a logical reason that links two contiguous stumps.**



Each stump is made by **taking the previous stumps mistakes into account** and try to remedy them.
How? By applying an importance weight on each sample : **missclassified points are given a higher weight** so that the next stumps are encouraged to focus on them.


**Stumps have more to say in the classification if they have been better at classifying in general!** 

![ADA](../images/ADA.png)

The voting weight for the stump $h_{t}$ is of the form $\alpha_{t}=L*log(\frac{1 - total.weigthed.error_{t}}{total.weigthed.error_{t}})$, where $L$ is the **learning rate**.

> NB : If a specific stump misclassifies 100% of the data it is actually a really good classifier if you decide to just reverse the labels...


The **learning rate** represents how much does the new iteration can deviate from the previous one. This is our new hyperparameter.


The update of the importance of individual points is following the the same formula as voting and then put in an exponential to get the updated weights:

$w^{t+1}_{i}=w^{t}_{i}e^{\alpha_{t+1}}$ if $h_{t+1}$ made a mistake for point $i$

> Those weights are then normalized.

And at the end classification is made following a weighted average of the stumps:

$G(x)=\sum^{T}_{t=1}\alpha_{t}G_{t}(x)$


### Hyperparameters

Since we are only using stumps we get rid of all the other hyperparameters describing the tree topology (max depth, min samples split, min leaf split ).

The only one left are the number of stumps (`n_estimators`) and the learning rate (`learning_rate`).

In [ ]:
from utils import contour_ADA
contour_ADA(X_3, y_3,
             n_estimators = 20 ,
             learning_rate = 0.1) #20 stumps, learning rate is 0.1

In [ ]:
contour_ADA(X_3, y_3, n_estimators = 20 , learning_rate= 1) #learning rate is 1

In [ ]:
%%time
l_rate=np.logspace(-2,1,30) 

fig,ax = plt.subplots(figsize=(10,7))

n_estimator=[5,10,20,50]
for i,ne in enumerate( n_estimator ):
    scores_valid = []

    for k in l_rate:
        tree = AdaBoostClassifier(n_estimators=ne,learning_rate=k , algorithm="SAMME")
        tree.fit(X_3,y_3)
        
        y_valid_pred = tree.predict(X_3_valid)

        scores_valid.append(accuracy_score(y_3_valid,y_valid_pred))
        
    ax.plot(l_rate, scores_valid,linewidth=3,label='N_estimators= '+str(ne))

ax.set_xlabel('Learning rate')
ax.set_ylabel('accuracy')
ax.set_xscale('log')
ax.legend(loc='best')


The two hyperparameters number of of stumps and learning rate are actually interdependent. 

The law associating the two is not obvious but just keep in mind that **if the number of stumps is large you don't need a big learning rate.**

### Cancer dataset

In [ ]:
%%time
grid_values = {'n_estimators':np.arange(500,1500,250), 
               'learning_rate':np.logspace(-3,1,5) 
               }

grid_ADA_roc_auc = GridSearchCV(AdaBoostClassifier(algorithm="SAMME"), 
                             param_grid = grid_values, 
                             scoring='roc_auc',
                             n_jobs=-1)

grid_ADA_roc_auc.fit(X_cancer_train, y_cancer_train)

y_decision_fn_scores_roc_auc=grid_ADA_roc_auc.score(X_cancer_test,y_cancer_test)

print('Grid best parameter (max. roc_auc): ', grid_ADA_roc_auc.best_params_)
print('Grid best score (roc_auc): ', grid_ADA_roc_auc.best_score_)
print('Grid best parameter (max. roc_auc) model on test: ', y_decision_fn_scores_roc_auc)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

y_pred_test_c=grid_ADA_roc_auc.predict(X_cancer_test)

    
bestNE = grid_ADA_roc_auc.best_params_["n_estimators"]
bestLR = grid_ADA_roc_auc.best_params_["learning_rate"]

plotTitle = """ADA boost - number of estimators: {}
learning rate: {}
Accuracy: {:.3f}""".format(bestNE,bestLR,
                           accuracy_score(y_cancer_test,y_pred_test_c) )

confusion_m_cancer = confusion_matrix(y_cancer_test, y_pred_test_c)

plt.figure(figsize=(5.5,4))
sns.heatmap(confusion_m_cancer, annot=True , 
            xticklabels=['benign','malignant'], 
            yticklabels=['benign','malignant'])
plt.title('test {} : {:.3f}'.format( "accuracy" , 
                                    accuracy_score(y_cancer_test,y_pred_test_c)  ))
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
RocCurveDisplay.from_estimator(grid_ADA_roc_auc,X_cancer_test, y_cancer_test)

[Back to ToC](#toc)

# Boosted gradient <a class="anchor" id="Boosted-Gradient"></a>

As you can already imagine from the name, in boosted gradient the 
optimization algorithm follows the gradient of a loss function (or a likelihood function!).

Just like ADAboost and random forest, boosted gradient **creates many trees and datasets**. 

In oposition to ADAboost each tree (not limited to stumps this time) is **fitted on the error from the previous tree** rather than a re-weighted dataset.

In the end you have a quorum of trees for which the **voting is done proportionally to the gradient value.**

![boosted](../images/boosted.png)


Boosted **gradient** refers to the gradient of a **loss function**.

When performing classification, the algorithm transforms the leaves scores into **log-odds ratio**, thus internally **switching from a classification problem to a regression one**.

This new perspective enables highly non linear classification rules, based on logistic regression.


Then, from tree to tree (ie. iterations), the algorithm tries to minimize the difference between the predicted log odds and its labeled values (1 or 0).

> This method is naturally adapted to regression as well, given that when we do a classification we actually have to transform into a regression problem.

#### hyperparameters

* single tree hyperparameters : 
    - Max Tree depth : `max_depth`
    - Min Splitting of nodes : `min_samples_split`
    - Min Splitting of leaves : `min_samples_leaf`
    - Max number of features : `max_features`
* number of iterations : `n_estimators`
* learning rate `learning_rate`


In [ ]:
from utils import contour_BG
contour_BG(X_3,y_3,
            n_estimators=100,
            learning_rate=0.01,
            max_depth=4,min_samples_split=20,min_samples_leaf=10,max_features='sqrt')

In [ ]:
## more iterations
contour_BG(X_3,y_3,
            n_estimators=1000,
            learning_rate=0.01,
            max_depth=4,min_samples_split=20,min_samples_leaf=10,max_features='sqrt')

## Boosted gradient on the penguin dataset

In [ ]:
%%time
grid_values = {'learning_rate':np.logspace(-1,1,5),
                'n_estimators':np.arange(100,501,100), 
               'max_depth':[2,3],
               'min_samples_split':[20],
              'min_samples_leaf':[10]}

grid_boosted_roc_auc = GridSearchCV(GradientBoostingClassifier(), 
                             param_grid = grid_values, 
                             scoring='roc_auc_ovr_weighted',n_jobs=-1)

grid_boosted_roc_auc.fit(X_penguin_train, y_penguin_train)

y_decision_fn_scores_roc_auc=grid_boosted_roc_auc.score(X_penguin_test, y_penguin_test)

print('Grid best parameter (max. roc_auc_ovr_weighted): ', grid_boosted_roc_auc.best_params_)
print('Grid best score (roc_auc_ovr_weighted): ', grid_boosted_roc_auc.best_score_)
print('Grid best parameter (max. roc_auc_ovr_weighted) model on test: ', y_decision_fn_scores_roc_auc)

In [ ]:
## predicting the labels on the test set    
y_pred_test=grid_boosted_roc_auc.predict(X_penguin_test)

confusion_m = confusion_matrix(y_penguin_test, y_pred_test)

plt.figure(figsize=(5.5,4))
sns.heatmap(confusion_m, annot=True , 
            xticklabels=['Adelie','Chinstrap','Gentoo'], 
            yticklabels=['Adelie','Chinstrap','Gentoo'])
plt.title('test {} : {:.3f}'.format( "accuracy" , 
                                    accuracy_score(y_penguin_test,y_pred_test)  ))
plt.ylabel('True label')
plt.xlabel('Predicted label')




In [ ]:
BG = GradientBoostingClassifier(learning_rate=grid_boosted_roc_auc.best_params_['learning_rate'],
                              n_estimators=grid_boosted_roc_auc.best_params_['n_estimators'],
                             max_depth=grid_boosted_roc_auc.best_params_['max_depth'],
                             min_samples_leaf=grid_boosted_roc_auc.best_params_['min_samples_leaf'],
                             min_samples_split=grid_boosted_roc_auc.best_params_['min_samples_split'])
BG.fit(X_penguin_train, y_penguin_train)
w=BG.feature_importances_#get the weights


featureW = pd.DataFrame( {'feature': X_penguin.columns,'weight':w} )

# sort them by absolute value
featureWsorted = featureW.sort_values(by=['weight'] , 
                                      ascending=False , 
                                      key=lambda col : col.abs())

# get the non-null ones
print('Features sorted per importance:')
featureWsorted.loc[ featureWsorted["weight"] !=0 ]